In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csic-2010-web-application-attacks/csic_database.csv
/kaggle/input/data-capec-multilabel/capec_mutilabel_new.csv
/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv


In [2]:
!git clone https://github.com/Phuongnguyen2710/NT213.O21.ANTT.git

fatal: destination path 'NT213.O21.ANTT' already exists and is not an empty directory.


In [3]:
%ls

NT213.O21.ANTT/


# **CAPEC DATASET**

## **Preprocessing data**

In [4]:
CAPEC_data = pd.read_csv("/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv")

/tmp/ipykernel_1202/264519686.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  CAPEC_data = pd.read_csv("/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv")


In [5]:
from urllib.parse import unquote_plus

def decode_column(df, column_name):
    """
    Decode values in the specified column of a DataFrame.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame.
        column_name (str): The name of the column to decode.
        
    Returns:
        pandas.DataFrame: A copy of the DataFrame with the specified column decoded.
    """
    def safe_decode(value):
        if isinstance(value, str):  # Kiểm tra nếu value là một chuỗi
            return unquote_plus(value)
        else:
            return value
    
    decoded_values = df[column_name].apply(safe_decode)
    df_decoded = df.copy()
    df_decoded[column_name] = decoded_values
    return df_decoded

In [6]:
CAPEC_data = decode_column(CAPEC_data,"request_body")

In [7]:
CAPEC_data['subpath'] = CAPEC_data['request_http_request']+CAPEC_data['request_body'].astype(str).apply(lambda x: "" if x == "nan" else '?'+ x)

In [8]:
CAPEC_data['subpath']

0                                                         /
1         /blog/index.php/2020/04/04/voluptatum-reprehen...
2                                      /blog/xmlrpc.php?rsd
3                                                         /
4         /blog/index.php/2020/04/04/nihil-tenetur-et-ad...
                                ...                        
907810                                                    /
907811                                                    /
907812                                                    /
907813                                                    /
907814                                                    /
Name: subpath, Length: 907815, dtype: object

In [9]:
CAPEC_data_new = CAPEC_data['request_http_request']
CAPEC_data_new = CAPEC_data[['request_http_request', 'request_body', 'subpath', '000 - Normal']].copy()
CAPEC_data_new = CAPEC_data_new[CAPEC_data_new['subpath'] != '/'].copy()
CAPEC_data_new = CAPEC_data_new.rename(columns={"000 - Normal": "label"})
CAPEC_data_new["label"].value_counts()

label
0    382299
1    226509
Name: count, dtype: int64

In [10]:
# Training : Validation : Test = 7.5 : 1.5 : 1.0
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(CAPEC_data_new['subpath'], CAPEC_data_new['label'], test_size=1 - train_ratio, random_state=42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 


print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)

print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)

print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (456606,)
y_train shape (456606,)
X_val shape (91321,)
y_val shape (91321,)
X_test shape (60881,)
y_test shape (60881,)


In [11]:
import urllib.parse

def url_decode(encoded_URL):
    """
    Decode the URL encoded string.
    
    Parameters:
        encoded_URL (str): The URL encoded string.
        
    Returns:
        str: The decoded URL string.
    """
    if isinstance(encoded_URL, str):  # Kiểm tra nếu tham số là một chuỗi
        if encoded_URL != "/":  # Kiểm tra nếu giá trị không phải là "/"
            decoded_url = urllib.parse.unquote(encoded_URL)
            return decoded_url
        else:
            return encoded_URL  # Trả về giá trị gốc nếu là "/"
    else:
        # Trả về None hoặc thông báo lỗi tùy vào yêu cầu cụ thể của bạn
        return None  # Hoặc raise TypeError("Invalid input type: expected str, got {type(encoded_URL)}")

In [12]:
def load_data(urls, max_length=1000):
    for url in urls:
        if url != '/':  # Kiểm tra nếu giá trị không phải là '/'
            try:
                url_lower = str(url).lower()
            except AttributeError:  # Bắt ngoại lệ nếu giá trị không thể chuyển đổi thành chuỗi
                print(f"Skipping lowercase conversion for value: {url}")
                url_lower = url
        else:
            url_lower = url  # Giữ nguyên giá trị nếu là '/'
    url_list = []
    for url in urls:
        # url decode
        decoded_url = url_decode(url)
        # unicode encode
        encoded_url = [ord(x) for x in str(decoded_url).strip()]
        encoded_url = encoded_url[:max_length]
        url_len = len(encoded_url)
        if url_len < max_length:
            # zero padding
            encoded_url += ([0] * (max_length - url_len))
        url_list.append((encoded_url))
    # convert to numpy array
    url_list = np.array(url_list)
    return url_list

In [13]:
train_data = load_data(X_train)
val_data = load_data(X_val)
test_data = load_data(X_test)

In [14]:
def create_label(labels):
  class_list = [x for x in labels]
  classes = np.array(class_list)
  return classes

In [15]:
train_classes = create_label(y_train)
val_classes = create_label(y_val)
test_classes = create_label(y_test)

## **Evaluate model**

### Architect A

In [16]:
model_paths = ['/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase2/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase3/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase4/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase5/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase6/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase7/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectA/Testcase8/model.h5']

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import keras
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix


2024-05-28 14:58:50.041827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 14:58:50.041894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 14:58:50.043626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [18]:
import time

results = []

for model_path in model_paths:
    # Load model
    model = load_model(model_path)
    
    y_true = test_classes
    # Calculate confusion matrix
    start_time = time.time()
    y_pred = model.predict(test_data)
    end_time = time.time()
    processing_time = end_time - start_time
    avg_processing_time = processing_time / len(test_data)

    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    cm = confusion_matrix(y_true, y_pred_binary)

    # Calculate TPR and FPR
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    # Trích xuất tên Testcase2 từ đường dẫn
    model_name = os.path.basename(os.path.dirname(model_path))

    # Lưu trữ kết quả
    results.append({
        'Case': model_name,
        'TPR': tpr,
        'FPR': fpr,
        'Avg Processing Time (s)': avg_processing_time
    })

# Tạo bảng kết quả
df = pd.DataFrame(results)
print(df)

1903/1903 ━━━━━━━━━━━━━━━━━━━━ 36s 19ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 50s 26ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 72s 38ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 85s 44ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 97s 51ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 109s 57ms/step
        Case       TPR       FPR  Avg Processing Time (s)
0  Testcase2  0.992822  0.039242                 0.000612
1  Testcase3  0.988462  0.037435                 0.000836
2  Testcase4  0.974458  0.031619                 0.001003
3  Testcase5  0.993659  0.039976                 0.001201
4  Testcase6  0.989079  0.037382                 0.001413
5  Testcase7  0.991677  0.038352                 0.001612
6  Testcase8  0.980139  0.034396                 0.001814


### Architect B

In [19]:
model_paths = ['/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectB/Test case [2,3,4,5]/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectB/Test case [3,4,5,6]/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectB/Test case [4,5,6,7]/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CAPEC-dataset/ArchitectB/Test case [5,6,7,8]/model.h5']

In [20]:
import time

results = []

for model_path in model_paths:
    # Load model
    model = load_model(model_path)
    
    y_true = test_classes
    # Calculate confusion matrix
    start_time = time.time()
    y_pred = model.predict(test_data)
    end_time = time.time()
    processing_time = end_time - start_time
    avg_processing_time = processing_time / len(test_data)

    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    cm = confusion_matrix(y_true, y_pred_binary)

    # Calculate TPR and FPR
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    # Trích xuất tên Testcase2 từ đường dẫn
    model_name = os.path.basename(os.path.dirname(model_path))

    # Lưu trữ kết quả
    results.append({
        'Case': model_name,
        'TPR': tpr,
        'FPR': fpr,
        'Avg Processing Time (s)': avg_processing_time
    })

# Tạo bảng kết quả
df = pd.DataFrame(results)
print(df)

1903/1903 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step
1903/1903 ━━━━━━━━━━━━━━━━━━━━ 60s 31ms/step
                  Case       TPR       FPR  Avg Processing Time (s)
0  Test case [2,3,4,5]  0.997358  0.044377                 0.001008
1  Test case [3,4,5,6]  0.969878  0.031672                 0.001005
2  Test case [4,5,6,7]  0.988286  0.037618                 0.001007
3  Test case [5,6,7,8]  0.985864  0.036335                 0.001008


# **CSIC-2010 DATASET**

## **Preprocessing data**

In [21]:
CSIC_2010_data = pd.read_csv('/kaggle/input/csic-2010-web-application-attacks/csic_database.csv')

In [22]:
CSIC_2010_data = CSIC_2010_data.rename(columns={'Unnamed: 0':'Target'})
CSIC_2010_data.head(5)

,Target,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1
1,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...
2,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.j...
3,Normal,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...
4,Normal,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...


In [23]:
CSIC_2010_data['Target'] = CSIC_2010_data['Target'].apply(lambda x:0 if str(x) == 'Normal' else 1)
CSIC_2010_data.head()

,Target,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp HTTP/1.1
1,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...
2,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.j...
3,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...
4,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...


In [24]:
#Xóa http1.1 cuối url
CSIC_2010_data['URL'] = CSIC_2010_data['URL'].str[:-8]
CSIC_2010_data.head(5)

,Target,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,classification,URL
0,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=1F767F17239C9B670A39E9B10C3825F4,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/index.jsp
1,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=81761ACA043B0E6014CA42A4BCD06AB5,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/anadir.j...
2,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=933185092E0B668B90676E0A2B0767AF,application/x-www-form-urlencoded,Connection: close,Content-Length: 68,id=3&nombre=Vino+Rioja&precio=100&cantidad=55&...,0,http://localhost:8080/tienda1/publico/anadir.jsp
3,0,GET,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=8FA18BA82C5336D03D3A8AFA3E68CBB0,NaN,close,NaN,NaN,0,http://localhost:8080/tienda1/publico/autentic...
4,0,POST,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,JSESSIONID=7104E6C68A6BCF1423DAE990CE49FEE2,application/x-www-form-urlencoded,Connection: close,Content-Length: 63,modo=entrar&login=choong&pwd=d1se3ci%F3n&remem...,0,http://localhost:8080/tienda1/publico/autentic...


In [25]:
# Training : Validation : Test = 7.5 : 1.5 : 1.0
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(CSIC_2010_data['URL'], CSIC_2010_data['Target'], test_size=1 - train_ratio, random_state=42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 


print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)

print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)

print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (45798,)
y_train shape (45798,)
X_val shape (9160,)
y_val shape (9160,)
X_test shape (6107,)
y_test shape (6107,)


In [26]:
#url decoder
import urllib.parse
def url_decode(encoded_URL):
  return urllib.parse.unquote(encoded_URL)

In [27]:
def load_data(urls, max_length=1000):
    urls = [s.lower() for s in urls]
    url_list = []
    for url in urls:
        # url decode
        decoded_url = url_decode(url)
        # unicode encode
        encoded_url = [ord(x) for x in str(decoded_url).strip()]
        encoded_url = encoded_url[:max_length]
        url_len = len(encoded_url)
        if url_len < max_length:
            # zero padding
            encoded_url += ([0] * (max_length - url_len))
        url_list.append((encoded_url))
    # convert to numpy array
    url_list = np.array(url_list)
    return url_list

In [28]:
train_data = load_data(X_train)
val_data = load_data(X_val)
test_data = load_data(X_test)

In [29]:
def create_label(labels):
  class_list = [x for x in labels]
  classes = np.array(class_list)
  return classes

In [30]:
train_classes = create_label(y_train)
val_classes = create_label(y_val)
test_classes = create_label(y_test)

## **Evaluate model**

### **Architect A**

In [31]:
%cd /kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase2

/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase2


In [32]:
model_paths = ['/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase2/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase3/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase4/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase5/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase6/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase7/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture A/TestCase8/model.h5']

In [33]:
import time

results = []

for model_path in model_paths:
    # Load model
    model = load_model(model_path)
    
    y_true = test_classes
    # Calculate confusion matrix
    start_time = time.time()
    y_pred = model.predict(test_data)
    end_time = time.time()
    processing_time = end_time - start_time
    avg_processing_time = processing_time / len(test_data)

    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    cm = confusion_matrix(y_true, y_pred_binary)

    # Calculate TPR and FPR
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    # Trích xuất tên Testcase2 từ đường dẫn
    model_name = os.path.basename(os.path.dirname(model_path))

    # Lưu trữ kết quả
    results.append({
        'Case': model_name,
        'TPR': tpr,
        'FPR': fpr,
        'Avg Processing Time (s)': avg_processing_time
    })

# Tạo bảng kết quả
df = pd.DataFrame(results)
print(df)

191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step
        Case       TPR       FPR  Avg Processing Time (s)
0  TestCase2  0.987905  0.235327                 0.000647
1  TestCase3  0.873976  0.139108                 0.000866
2  TestCase4  0.591885  0.001129                 0.001042
3  TestCase5  0.870854  0.139108                 0.001263
4  TestCase6  0.988295  0.229120                 0.001430
5  TestCase7  0.873586  0.138826                 0.001625
6  TestCase8  0.587593  0.000847                 0.001891


### **Architect B**

In [34]:
model_paths = ['/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture B/TestCase[2,3,4,5]/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture B/TestCase[3,4,5,6]/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture B/TestCase[4,5,6,7]_new/model.h5',
               '/kaggle/working/NT213.O21.ANTT/CSIC-2010-dataset/Architecture B/TestCase[5,6,7,8]/model.h5']

In [35]:
import time

results = []

for model_path in model_paths:
    # Load model
    model = load_model(model_path)
    
    y_true = test_classes
    # Calculate confusion matrix
    start_time = time.time()
    y_pred = model.predict(test_data)
    end_time = time.time()
    processing_time = end_time - start_time
    avg_processing_time = processing_time / len(test_data)

    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    cm = confusion_matrix(y_true, y_pred_binary)

    # Calculate TPR and FPR
    tn, fp, fn, tp = cm.ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    # Trích xuất tên Testcase2 từ đường dẫn
    model_name = os.path.basename(os.path.dirname(model_path))

    # Lưu trữ kết quả
    results.append({
        'Case': model_name,
        'TPR': tpr,
        'FPR': fpr,
        'Avg Processing Time (s)': avg_processing_time
    })

# Tạo bảng kết quả
df = pd.DataFrame(results)
print(df)

191/191 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 126ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 29s 148ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step
                    Case       TPR       FPR  Avg Processing Time (s)
0      TestCase[2,3,4,5]  0.990636  0.228837                 0.003116
1      TestCase[3,4,5,6]  0.873195  0.139108                 0.003988
2  TestCase[4,5,6,7]_new  0.991026  0.231941                 0.004694
3      TestCase[5,6,7,8]  0.912993  0.167325                 0.005430
